# The implementing of a Tic-Tac-Toe using of all AI Techniques: Minimax, Alpha-Beta Pruning, and Q-Learning

## Instructions on playing this game

#### Start the Game: Make Your Move by select a number from 1 to 9 corresponding to the grid cells, from top-left to bottom-right, to place your marker (X).
#### Understand the AI: You're playing against an advanced AI that learns and predicts optimal moves using combination of Minimax, Alpha-Beta Pruning, and Q-Learning techniques.
#### Play to Win: Aim to align three of your markers (X) in a row, column, or diagonal to win.
#### Game End: The game ends when one player wins by aligning three markers or when all grid spaces are filled, resulting in a draw.

In [4]:
from random import uniform, choice
import numpy as np

board = [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]

def Gameboard(board):
    chars = {1: 'X', -1: 'O', 0: ' '}
    for x in board:
        print('|', end='')
        for y in x:
            ch = chars[y]
            print(f' {ch} |', end='')
        print('\n')



def Clearboard(board):
    for x, row in enumerate(board):
        for y, col in enumerate(row):
            board[x][y] = 0

def winningPlayer(board, player):
    conditions = [[board[0][0], board[0][1], board[0][2]],
                     [board[1][0], board[1][1], board[1][2]],
                     [board[2][0], board[2][1], board[2][2]],
                     [board[0][0], board[1][0], board[2][0]],
                     [board[0][1], board[1][1], board[2][1]],
                     [board[0][2], board[1][2], board[2][2]],
                     [board[0][0], board[1][1], board[2][2]],
                     [board[0][2], board[1][1], board[2][0]]]

    if [player, player, player] in conditions:
        return True

    return False

def gameWon(board):
    return winningPlayer(board, 1) or winningPlayer(board, -1)

def printResult(board):
    if winningPlayer(board, 1):
        print('You have won!, Min Max  ' + '\n')

    elif winningPlayer(board, -1):
        print('AI has won!, Alpha Beta Pruning  ' + '\n')

    else:
        print("It's a draw!" + '\n')

def blanks(board):
    blank = []
    for x, row in enumerate(board):
        for y, col in enumerate(row):
            if board[x][y] == 0:
                blank.append((x, y))

    return blank

def boardFull(board):
    if len(blanks(board)) == 0:
        return True
    return False

def setMove(board, x, y, player):
    board[x][y] = player

def playerMove(board):
    moves = {1: (0, 0), 2: (0, 1), 3: (0, 2),
             4: (1, 0), 5: (1, 1), 6: (1, 2),
             7: (2, 0), 8: (2, 1), 9: (2, 2)}
    while True:
        try:
            move = int(input('Enter a number between 1-9: '))
            if move < 1 or move > 9:
                print('Invalid Move! Try again!')
            elif moves[move] not in blanks(board):
                print('Invalid Move! Try again!')
            else:
                row, col = moves[move]
                setMove(board, row, col, 1)
                Gameboard(board)
                break
        except ValueError:
            print('Enter a number!')

def getScore(board):
    if winningPlayer(board, 1):
        return 10
    elif winningPlayer(board, -1):
        return -10
    else:
        return 0

class QLearningPlayer():
    def __init__(self):
        self.name = 'Q-Learning'
        self.q = {}
        self.init_q = 1 # "optimistic" 1.0 initial values
        self.lr = 0.3
        self.gamma = 0.9
        self.epsilon = 1.0
        self.max_epsilon = 1.0
        self.min_epsilon = 0.01
        self.decay_rate = 0.01
        self.action_n = 9
        self.win_n = 0
        self.last_state = (' ',) * 9
        self.last_action = -1

    def action(self, state, actions):
        state = tuple(state)
        self.last_state = state
        if uniform(0, 1) > self.epsilon and self.q.get(state):
            i = np.argmax([self.q[state][a] for a in actions])
            action = actions[i]
        else:
            action = choice(actions)
        self.last_action = action
        return action

    def reward(self, reward, state):
        if self.last_action >= 0:
            if reward == 1:
                self.win_n += 1
            state = tuple(state)
            if self.q.get(self.last_state):
                q = self.q[self.last_state][self.last_action]
            else:
                self.q[self.last_state] = [self.init_q] * self.action_n
                q = self.init_q
            self.q[self.last_state][self.last_action] = q + self.lr * (reward + self.gamma * np.max(self.q.get(state, [self.init_q]*self.action_n)) - q)

    def episode_end(self, episode):
        self.epsilon = self.min_epsilon + (self.max_epsilon - self.min_epsilon) * np.exp(-self.decay_rate*(episode+1))

    def print_q(self):
        for k,v in self.q.items():
            print(k,v)

def makeQLearningMove(board, qplayer):
    state = [item for sublist in board for item in sublist]
    if np.count_nonzero(state) == 0:
        # If it's the first move, choose a random available position
        available_positions = blanks(board)
        random_position = choice(available_positions)
        setMove(board, random_position[0], random_position[1], -1)
    else:
        # For subsequent moves, use Q-learning to select the action
        while True:
            action = qplayer.action(state, range(len(state)))
            row, col = action // 3, action % 3
            if board[row][col] == 0:
                setMove(board, row, col, -1)
                break


if __name__ == "__main__":
    print("Initial Game Board:")
    Gameboard(board)
    print("Playing Tic-Tac-Toe:")
    q_player = QLearningPlayer()
    episode = 0
    while not (boardFull(board) or gameWon(board)):
        playerMove(board)
        if gameWon(board) or boardFull(board):
            break
        print("AI is making a move")
        makeQLearningMove(board, q_player)
        q_player.reward(getScore(board), [item for sublist in board for item in sublist])
        episode += 1
        if gameWon(board) or boardFull(board):
            break
        Gameboard(board)  # Print the board once after both moves

    printResult(board)  # Print the game result
    Gameboard(board)  # Print the final state of the board

Initial Game Board:
|   |   |   |

|   |   |   |

|   |   |   |

Playing Tic-Tac-Toe:
Enter a number between 1-9: 1
| X |   |   |

|   |   |   |

|   |   |   |

AI is making a move
| X |   |   |

|   |   |   |

|   | O |   |

Enter a number between 1-9: 9
| X |   |   |

|   |   |   |

|   | O | X |

AI is making a move
| X |   | O |

|   |   |   |

|   | O | X |

Enter a number between 1-9: 2
| X | X | O |

|   |   |   |

|   | O | X |

AI is making a move
| X | X | O |

| O |   |   |

|   | O | X |

Enter a number between 1-9: 7
| X | X | O |

| O |   |   |

| X | O | X |

AI is making a move
| X | X | O |

| O | O |   |

| X | O | X |

Enter a number between 1-9: 6
| X | X | O |

| O | O | X |

| X | O | X |

It's a draw!

| X | X | O |

| O | O | X |

| X | O | X |



### Breaking down the code step-by-step:

The game is played on a 3x3 grid where '1' represents the human player (X), and '-1' represents the AI player (O).

Functions and Classes:

Gameboard(board): Prints the current state of the board. Each cell is displayed based on its value: 'X' for 1, 'O' for -1, and ' ' for 0.

Clearboard(board): Resets the board to all zeros, indicating an empty board.

winningPlayer(board, player): Checks if the specified player has won the game by checking all possible winning combinations on the board.

gameWon(board): Returns True if either player has won the game.

printResult(board): Prints the result of the game based on whether a player has won or if the game ended in a draw.

blanks(board): Returns a list of tuples representing the coordinates of all empty cells on the board.

boardFull(board): Checks if the board is full.

setMove(board, x, y, player): Places a player's mark on the board at the specified coordinates.

playerMove(board): Handles the human player's move based on input from the console. It ensures valid moves are made.

QLearningPlayer: A class implementing the Q-learning algorithm for the AI:
action(state, actions): Determines the AI's next move using an ε-greedy strategy,
reward(reward, state): Updates the Q-values based on the received reward and the current state,
episode_end(episode): Adjusts the exploration rate (ε) over time to balance exploration and exploitation,
print_q(): Prints the current Q-values for debugging purposes.

makeQLearningMove(board, qplayer): Determines the AI's move using the Q-learning algorithm. It selects an action based on the current state and updates the board.

Main Game Loop
Initializes the game board.
Alternates turns between the human player and the AI until the game is won or the board is full.
After each move, updates the board state and checks for a win or draw.
Prints the game's outcome and the final state of the board.

The code essentially sets up a game of Tic-Tac-Toe where the human player interacts through the console, and the AI uses a rudimentary combination approach to make decisions. The AI's learning mechanism allows it to adjust its strategy based on past experiences (albeit limited in this code to the current game session without long-term memory). The game flow is controlled through a while loop that continues until there's a win or the board is full, with feedback provided on the console for each move.